<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ServiceNow Multipath Pattern Analysis
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>ServiceNow is a widely-used platform for managing IT service requests and incidents. While it effectively captures case-level data and status transitions, it lacks native capabilities to perform deep-dive pattern analysis across the lifecycle of a case — especially in terms of how cases move through different support groups, how long they stay in each state, and what behaviors lead to resolutions or escalations. Due to limitations in ServiceNow's native analytics capabilities, answering questions like below are extremely difficult :<ul style = 'font-size:16px;font-family:Arial'>
    <li>How many cases moved from L1 to L2 multiple times?</li>
    <li>Which cases were resolved in a single touch by L2 teams?</li>
    <li>Which assignment groups handled the highest severity incidents before resolution?</li>
    </ul>
<p style = 'font-size:18px;font-family:Arial'><b>Business Value</b></p>
<p style = 'font-size:16px;font-family:Arial'>By integrating ServiceNow case audit data with Vantage's nPath® functionality we can unlock critical insights from the case history:<ul style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'><b>Operational Insights </b>like Understand common and rare resolution paths, identify escalation trends from L1 to L2 or other teams, discover patterns in delayed resolutions or reassignment loops</li>
    <li style = 'font-size:16px;font-family:Arial'><b>Improved Efficiency and SLA Management</b> like Highlight cases that violate SLAs due to excessive handoffs, spot 'one-touch' resolution patterns for knowledge reuse, enable proactive case routing strategies.</li>
    <li style = 'font-size:16px;font-family:Arial'><b>Enhanced Reporting and Decision-Making </b>like Weekly trend analysis for leadership using dashboards, support data-driven staffing decisions by understanding group-wise workloads.</li>
    </ul>
</p>
<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial'>Vantage has unique analytic capabilities for  looking for pattern. nPath® enables sequence mining and event pattern detection across logs or audit trails using syntax similar to regex, making it ideal for modeling complex case journeys. Teradata can handle large volumes of ServiceNow audit logs efficiently, enabling transformation of raw audit data into enriched analytic tables without compromising performance.</p> 


<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage,  import python packages and explore the dataset</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start with importing the required libraries, set environment variables and connect to Vantage.</p>

In [ ]:
#import libraries
import getpass
import warnings

from teradataml import *
import matplotlib.pyplot as plt


display.max_rows = 5

warnings.filterwarnings('ignore')

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys. </p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=FF_ServiceNow_Multipath_PatternAnalysis_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys.</p>

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.
</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_ServiceNow_cloud');"
# takes about 20sec minute, estimated space: 0 MB
# %run -i ../run_procedure.py "call get_data('DEMO_ServiceNow_local');"
# takes about 1 minute, estimated space: 20 MB

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Analyze the raw data set</b></p>
<p style = 'font-size:16px;font-family:Arial'> Sys audit table in ServiceNow stores the assignment groups and states but it stores the changes in assignment group only but it doesnt store the first assignment group, hence we built a raw data table. It's a mix of different tables (sys_audit, sys_user etc) in ServiceNow that allows us to identify what we think are the most important parts for that we need for the analysis of a journey.<br>One of the column of interest is the audit time stamp. This is the time stamp of when an event occurred. There are two very different dates in the table, the date of when a case was created and then the date of when something happened. We also have column for here how long it has happened since the since the case or incident was created (hrs_elapsed). Table also captures the highest severity of the tickets. Let us take a look at the data in the table.</p> 

In [ ]:
tdf= DataFrame(in_schema('DEMO_ServiceNow', 'Audit_Raw_Data'))
tdf

<p style = 'font-size:16px;font-family:Arial'>We can check data for one of the tickets.</p>

In [ ]:
filtered_tdf = tdf[tdf['sn_ticket'] == 'INC0342089']
filtered_tdf.sort('sn_audit_TS')

<p style = 'font-size:16px;font-family:Arial'>If we look at the ticket data above we can see that the assignment group when the ticket was created was 'Cloud BaaS Operator' and later it was reassigned to 'GSO L1 Cloud Support'. There can be other tickets where this type of reassignment is done or there may be tickets where there is multiple reassignments. If we want to see the pattern or number of tickets where this happens we can easily do that using Vantage's nPath function.</p>
<p style = 'font-size:16px;font-family:Arial'>The nPath function scans a set of rows, looking for patterns that you specify. For each set of input rows that matches the pattern, nPath produces a single output row. The function provides a flexible pattern-matching capability that lets you specify complex patterns in the input data and define the values that are output for each matched input set.</p>

<p style = 'font-size:16px;font-family:Arial'>Now consider that we want to find out the tickets that were resolved by a particular assignment group for example GSO L1 team and we also want to check all the reassignments that were done before the resolution.<br> First let us check the values in the sn_ag column which holds the values for the assigment groups. </p>

In [ ]:
catsum = CategoricalSummary(data=tdf,
                             target_columns=['sn_ag']
                            )
 
catsum.result.sort('DistinctValue').head(20)

<p style = 'font-size:16px;font-family:Arial'>We will use nPath function and define the pattern according to our requirements. We also want to get the values if there is any ressignment done.</p>

In [ ]:
npath_gso_l1 = NPath(data1 = tdf, 
                      data1_partition_column = ['sn_ticket'], 
                      data1_order_column = 'sn_audit_TS', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['fieldname = \'resolved_by\' and sn_ag IN (\'GSO L1 Cloud Support\',\'GSO L1 EU Restricted\') AND sn_state IN (\'Resolved\') AS resolved_gso_l1',
                                 'TRUE as OTHER'], 
                      pattern = 'OTHER?*.resolved_gso_l1', 
                      result = ['FIRST (sn_ticket of resolved_gso_l1) as sn_ticket', 
                                'FIRST (sn_audit_TS of resolved_gso_l1) as resolved_gso_l1_ts',
                                'COUNT (DISTINCT  sn_ag of ANY(other,resolved_gso_l1) ) as sn_ags', 
                                'ACCUMULATE (DISTINCT  cast(sn_ag as VARCHAR(20)) of ANY(other,resolved_gso_l1)) as path' 
                               ])

npath_gso_l1.result

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>4. Analysis and Visualization</b>
<p style = 'font-size:16px;font-family:Arial'> We can perform analysis on the data in-database and visualize the results by plotting the graphs and paths.</p>

In [ ]:
d1 = npath_gso_l1.result.groupby(['path']).count()
d1= d1.assign(drop_columns = True,
              path = d1.path,
              ticket_count = d1.count_sn_ticket)
d1

In [ ]:
sn_ags_plot = npath_gso_l1.result.groupby(['sn_ags']).count().sort(['count_sn_ticket'], ascending = False)
sn_ags_plot

In [ ]:
plot =  sn_ags_plot.plot(x=sn_ags_plot.sn_ags, y=sn_ags_plot.count_path,
                                 kind='bar',xlabel = 'Number of Reassignments', yabel = 'Count of tickets', 
                                 heading="Reassignments done before ticket resolved by GSO L1", figsize=(600, 400))
 
# Display the plot.
plot.show()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>4.1  Sankey Charts</b></p>
<p style = 'font-size:16px;font-family:Arial'> In order to visualize the distribution of the different path of events, we typically use Sankey diagram of the aggregated over the paths reported by the NPATH command.


In [ ]:
from tdnpathviz.visualizations import plot_first_main_paths

In [ ]:
plot_first_main_paths(npath_gso_l1.result,path_column='path',id_column='sn_ticket')

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here in this notebook we have seen that with Teradata Vantage and ClearScape Analytics we can find patterns inside ServiceNow data. We can find common paths in our data and improve efficiency and quality of service we provide to ServiceNow customers.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Cleanup </b></p>
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ServiceNow');"   # Takes about 10 seconds, optional if you want to use the data later

In [ ]:
remove_context()

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset Information</b>
<p style = 'font-size:16px;font-family:Arial'>Dataset used here is simulated data based on the actual ServiceNow audit data.</p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2025 Teradata. All rights reserved.
        </div>
    </div>
</footer>